In [1]:
# Test Openai api with langchain is working
from langchain_openai import ChatOpenAI
import os

OpenAI_llm = ChatOpenAI(model="gpt-4o", temperature=0.6, api_key= os.environ['OPENAI_API_KEY'])
messages=[("Say hello if you are working")]
ai_msg = OpenAI_llm.invoke(messages)
ai_msg






AIMessage(content="Hello! I'm here and ready to help. What can I do for you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 13, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-3472316d-22a3-496c-8c97-a2fae56a3cc4-0', usage_metadata={'input_tokens': 13, 'output_tokens': 18, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# Testing agents with langchain
from langchain.agents import AgentType, initialize_agent, load_tools

tools = load_tools(["serpapi"], llm = OpenAI_llm)
agent = initialize_agent(tools, llm=OpenAI_llm, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION)

agent.run("What is current value of 1 dollar in rupees?")

C:\Users\Βασίλης\AppData\Local\Temp\ipykernel_9244\1254470714.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm=OpenAI_llm, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION)
C:\Users\Βασίλης\AppData\Local\Temp\ipykernel_9244\1254470714.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~i

'The current value of 1 US dollar is approximately 86.53 Indian Rupees.'

In [2]:
from config import constants 
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    api_key= constants.OPENAI_API_KEY, 
    temperature=0.6,
    model="gpt-4o"
)

ImportError: cannot import name 'constants' from 'config' (unknown location)